In [1]:
import pandas as pd

In [2]:
diabetes_data = pd.read_csv('dataset/PimaIndians_processed.csv')
diabetes_data.sample(10)

,pregnant,glucose,diastolic,triceps,insulin,bmi,diabetes,age,test
290,0.841505,-0.766596,-0.854419,-0.965953,-0.859854,-0.881393,-0.075485,0.111428,0
77,4.271153,1.309884,0.107109,1.128686,-0.354336,1.113281,0.851925,1.583784,1
138,0.529718,-0.474591,0.107109,1.319108,-0.682923,0.429393,-0.753653,0.209585,0
191,-1.029213,-0.961266,2.350673,0.938264,-0.708199,1.469473,1.443148,0.405899,0
6,-1.029213,-0.150141,1.068636,1.699951,0.623000,1.811417,0.081015,0.013271,1
0,-0.717427,-1.091046,-0.373655,-0.585110,-0.522842,-0.710421,-1.031876,-0.968299,0
283,-0.717427,1.342329,0.908382,1.319108,-0.750325,-0.040780,-0.527597,1.878255,0
226,-0.717427,-0.734151,0.107109,-1.632429,-0.725049,-1.109356,0.391118,-0.281200,0
169,1.776863,0.952989,0.587873,0.462210,0.125907,0.158687,1.072184,0.209585,1
369,-0.717427,-0.474591,-0.854419,1.604740,0.184884,0.343907,-0.313134,-0.673828,0


In [3]:
x = diabetes_data.drop('test', axis=1)
y= diabetes_data['test']

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train.shape, y_train.shape

((313, 8), (313,))

In [6]:
x_test.shape, y_test.shape

((79, 8), (79,))

In [8]:
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [9]:
log_clf = LogisticRegression(C=1, solver='liblinear')
svc_clf = SVC(C=1, kernel='linear', gamma='auto')
nav_clf = GaussianNB()

In [10]:
voting_clf_hard = VotingClassifier(
    estimators=[
        ('log_clf', log_clf),
        ('svc_clf', svc_clf),
        ('nav_clf', nav_clf)
    ],
    voting='hard'
)

In [11]:
voting_clf_hard.fit(x_train, y_train)

VotingClassifier(estimators=[('log_clf',
                              LogisticRegression(C=1, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('svc_clf',
                              SVC(C=1, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=0.0,
                                  decision_function_shape='ovr', degree=3,
                              

In [12]:
y_pred = voting_clf_hard.predict(x_test)

In [14]:
from sklearn.metrics import accuracy_score

print('Accuracy Score :', accuracy_score(y_test, y_pred))

Accuracy Score : 0.7974683544303798


In [17]:
for clf_hard in (log_clf, svc_clf, nav_clf, voting_clf_hard):
    clf_hard.fit(x_train, y_train)
    y_pred = clf_hard.predict(x_test)
    
    print(clf_hard.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.810126582278481
SVC 0.7848101265822784
GaussianNB 0.759493670886076
VotingClassifier 0.7974683544303798


In [18]:
svc_clf_soft = SVC(C=1, kernel='linear', gamma='auto', probability=True)
voting_clf_soft = VotingClassifier(
    estimators=[
        ('log_clf', log_clf),
        ('svc_clf_soft', svc_clf_soft),
        ('naive', nav_clf)
    ],
    voting = 'soft',
    weights = [0.25,0.5,0.25]
)

In [19]:
for clf_hard in (log_clf, svc_clf_soft, nav_clf, voting_clf_soft):
    clf_hard.fit(x_train, y_train)
    y_pred = clf_hard.predict(x_test)
    
    print(clf_hard.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.810126582278481
SVC 0.7848101265822784
GaussianNB 0.759493670886076
VotingClassifier 0.759493670886076
